In [1]:
import keras 
from keras.layers import Conv2D,MaxPooling2D,Flatten,Dense
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
import keras.utils.image_utils
import tensorflow as tf

In [2]:
model = Sequential()
model.add(Conv2D(32,(3,3),activation = 'relu', input_shape=(150,150,3)))
model.add(MaxPooling2D())
model.add(Conv2D(32,(3,3),activation = 'relu'))
model.add(MaxPooling2D())
model.add(Conv2D(32,(3,3),activation = 'relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(100,activation='relu'))
model.add(Dense(1,activation ='sigmoid'))

In [3]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics =['accuracy'])

In [5]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)

In [6]:
test_datagen = ImageDataGenerator(rescale = 1./255 )

In [7]:
training_set = train_datagen.flow_from_directory(
    r'C:\Users\AA\Documents\Dataset\train',
    target_size = (150,150),
    batch_size=16,
    class_mode = 'binary'
)

Found 4791 images belonging to 2 classes.


In [8]:
test_set = test_datagen.flow_from_directory(
    r'C:\Users\AA\Documents\Dataset\test',
    target_size = (150,150),
    batch_size=16,
    class_mode = 'binary'
)

Found 1197 images belonging to 2 classes.


In [9]:
history = model.fit_generator(
    training_set,
    epochs = 20,
    validation_data = test_set,
)

C:\Users\AA\AppData\Local\Temp\ipykernel_16072\1063576067.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/20
300/300 [==============================] - 142s 467ms/step - loss: 0.2065 - accuracy: 0.9132 - val_loss: 0.1266 - val_accuracy: 0.9457
Epoch 2/20
300/300 [==============================] - 134s 447ms/step - loss: 0.1120 - accuracy: 0.9616 - val_loss: 0.1639 - val_accuracy: 0.9365
Epoch 3/20
300/300 [==============================] - 131s 436ms/step - loss: 0.0779 - accuracy: 0.9739 - val_loss: 0.0779 - val_accuracy: 0.9749
Epoch 4/20
300/300 [==============================] - 134s 447ms/step - loss: 0.0655 - accuracy: 0.9768 - val_loss: 0.0737 - val_accuracy: 0.9741
Epoch 5/20
300/300 [==============================] - 136s 455ms/step - loss: 0.0505 - accuracy: 0.9843 - val_loss: 0.0615 - val_accuracy: 0.9749
Epoch 6/20
300/300 [==============================] - 143s 476ms/step - loss: 0.0533 - accuracy: 0.9818 - val_loss: 0.0690 - val_accuracy: 0.9749
Epoch 7/20
300/300 [==============================] - 149s 496ms/step - loss: 0.0497 - accuracy: 0.9827 - val_loss: 0.0583 -

In [10]:
model.save('Mask_Detector.h5',history)

In [2]:
import numpy as np
from keras.models import load_model
from keras.preprocessing import image 
import cv2
import keras.utils.image_utils
import tensorflow as tf

In [3]:
mymodel=load_model('Mask_Detector.h5')

In [6]:
mymodel.output_shape

(None, 1)

In [11]:
train_loss,train_acc = mymodel.evaluate(training_set,steps=16)

16/16 [==============================] - 4s 270ms/step - loss: 0.0182 - accuracy: 0.9922


In [4]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [5]:
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

while cap.isOpened():
    s, img=cap.read()
    face = face_cascade.detectMultiScale(img,scaleFactor=1.1,minNeighbors=4)
    
    for(x,y,w,h) in face:
        face_img = img[y:y+h,x:x+w]
        cv2.imwrite('temp.jpg',face_img)
        test_img = tf.keras.utils.load_img('temp.jpg',target_size=(150,150,3))
        test_img = tf.keras.utils.img_to_array(test_img)
        test_img = np.expand_dims(test_img, axis=0)
        
        pred = mymodel.predict(test_img)[0][0]
        
        if pred==1:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),2)
            cv2.rectangle(img,(x,y-40),(x+w,y),(0,0,255),-1)
            cv2.putText(img,"No Mask",(x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8,(255,255,255),2)
        else:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
            cv2.rectangle(img,(x,y-40),(x+w,y),(0,255,0),-1)
            cv2.putText(img,"Mask",(x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8,(255,255,255),2)
    cv2.imshow('Image',img)
    if cv2.waitKey(1) == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 47ms/step
